# Plot spike amino-acid differences between SARS-CoV-2 clades

This is a Jupyter notebook written by Jesse Bloom that plots spike amino-acid differences among SARS-CoV-2 Pango clades.
This notebook is found in the GitHub repo at [https://github.com/jbloom/SARS2-clade-spike-diffs](https://github.com/jbloom/SARS2-clade-spike-diffs), and can be run interactively using `mybinder` by going to [https://mybinder.org/v2/gh/jbloom/SARS2-clade-spike-diffs/main?labpath=spike-diffs.ipynb](https://mybinder.org/v2/gh/jbloom/SARS2-clade-spike-diffs/main?labpath=spike-diffs.ipynb).

It reads Pango clade definitions from the [clade definition JSON file](https://raw.githubusercontent.com/corneliusroemer/pango-sequences/main/data/pango-consensus-sequences_summary.json) maintained [by Cornelius Roemer](https://github.com/corneliusroemer/pango-sequences).

To use, specify the Pango clades you want to plot in the list below.
Then run the rest of the notebook, and it will create an interactive [altair](https://altair-viz.github.io/) plot of the differences.
There is an option at the bottom of this plot to show all sites that differ in any of the clades from the Wuhan-Hu-1 reference, or just sites that differ among the clades of interest.
You can download a SVG of the plot by clicking the circle with the three small dots in the upper right of the plot.
Note that a limitation of these plots is that they show substitutions and deletions, but **not** insertions.

In [1]:
# Specify the clades to plot. These must either be standard Pango names listed
# in Cornelius Roemer's JSON defintions OR specified in `custom_clades` below
clades = ["BA.2", "XBB.1.5", "HK.3.1", "JN.1", "BA.2.87.1"]

# Specify any clades not in the Pango site, as a dict giving the clade names followed
# by the space de-limited mutations in the form E484A. Below is an example.
# Note the clade is not plotted as it is not listed in `my_clades` above
custom_clades = {
    "my_clade": "Y453F E484A",
}

Read the Pango clade information:

In [2]:
import json
import urllib.request

import altair as alt

import pandas as pd

pango_json = "https://raw.githubusercontent.com/corneliusroemer/pango-sequences/main/data/pango-consensus-sequences_summary.json"
with urllib.request.urlopen(pango_json) as url:
    pango_clades = json.load(url)
    
if len(clades) != len(set(clades)):
    raise ValueError(f"duplicate clades in {clades}")

clade_spike_muts = {}
for clade in clades:
    if clade in pango_clades:
        clade_spike_muts[clade] = [
            m.split(":")[1]
            for m in pango_clades[clade]["aaSubstitutions"] + pango_clades[clade]["aaDeletions"]
            if m.startswith("S:")
        ]
        if clade in custom_clades:
            raise ValueError(f"{clade} is in {pango_json} and {my_clades}")
    elif clade in custom_clades:
        clade_spike_muts[clade] = custom_clades[clade].split()
    else:
        raise ValueError(f"Cannot find {clade=} in {pango_json=}")

Make a data frame of the mutations to plot:

In [3]:
def domain(r):
    if 13 <= r <= 305:
        return "NTD"
    elif 331 <= r <= 528:
        return "RBD"
    elif 682 <= r <= 1211:
        return "S2"
    else:
        return "other"

mutations = (
    pd.Series(clade_spike_muts, name="mutation")
    .rename_axis("clade")
    .reset_index()
    .explode("mutation")
    .assign(
        reference_aa=lambda x: x["mutation"].str[0],
        site=lambda x: x["mutation"].str[1: -1].astype(int),
        mutant_aa=lambda x: x["mutation"].str[-1],
        is_deletion=lambda x: x["mutant_aa"] == "-",
        reference_site=lambda x: x["reference_aa"] + x["site"].astype(str),
        domain=lambda x: x["site"].map(domain),
        differ_among_clades=lambda x: (
            x.groupby("site")["mutant_aa"].transform(lambda s: (len(s) != len(clades)) or (s.nunique() > 1))
        )
    )
)

assert len(mutations.groupby(["site", "reference_aa"])) == mutations["site"].nunique()

Draw the plot using `altair`.
Save a SVG by clicking on the three dots in the upper right of the plot below:

In [4]:
cell_width = 11

differ_among_clades = alt.selection_point(
    fields=["differ_among_clades"],
    bind=alt.binding_radio(
        options=[None, True],
        labels=["differ from Wuhan-Hu-1 reference in any of the clades", "differ among the clades"],
        name="show sites that:",
    ),
    value=None,
)

chart_base = alt.Chart(mutations).add_params(differ_among_clades).transform_filter(differ_among_clades)

heatmap_base = (
    chart_base
    .encode(
        alt.X("reference_site", sort=alt.SortField("site"), title="site (indicating amino-acid in Wuhan-Hu-1)"),
        alt.Y("clade", sort=list(clades)),
        alt.Fill("is_deletion", legend=None, scale=alt.Scale(scheme="set1")),
        alt.Text("mutant_aa"),
        tooltip=["site", "mutation", "clade"],
    )
    .properties(
        width=alt.Step(cell_width),
        height=alt.Step(15),
    )
)

domain_overlay = (
    chart_base
    .transform_calculate(y="'domain'")
    .encode(
        alt.X("reference_site", sort=alt.SortField("site"), title=None, axis=None),
        alt.Y("y:N", title=None),
        alt.Color(
            "domain",
            legend=alt.Legend(orient="top", titleOrient="left", offset=10),
            scale=alt.Scale(scheme="accent"),
        ),
        tooltip=["site", "reference_aa", "domain"],
    )
    .properties(
        width=alt.Step(cell_width),
        height=10,
    )
    .mark_rect(stroke="gray")
)

heatmap = heatmap_base.mark_rect(stroke="gray", fillOpacity=0.4)

aas = heatmap_base.mark_text(stroke="black", size=10)

chart = alt.vconcat(domain_overlay, (heatmap + aas), spacing=3).properties(
    title=alt.TitleParams(
        "Spike amino-acid differences among clades.",
        subtitle=[
            "Use option at bottom to show all sites that differ from the Wuhan-Hu-1 reference, or just sites that differ among plotted clades.",
            "For Jesse Bloom's code to make these plots, see https://github.com/jbloom/SARS2-clade-spike-diffs",
            "Clade sequences taken from Cornelius Roemer's definitions at https://github.com/corneliusroemer/pango-sequences",
            "Note that only substitutions and deletions but NOT insertions are are shown.",
            "Sites are numbered using Wuhan-Hu-1 reference numbering.",
        ],
        dy=-15,
    ),
)

chart

alt.VConcatChart(...)